In [2]:
import os
from dotenv import load_dotenv

import pandas as pd
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

d:\ProjectNLP\Anime\animeee\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [4]:
import os
from langchain.chat_models import init_chat_model
model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know. Parrots are known for mimicking human speech, right? But why do they do that? Maybe it\'s related to their social behavior. I remember reading somewhere that parrots are social animals, so maybe they talk to communicate with each other or with humans.\n\nI think about how other birds, like parrots, learn to mimic sounds. For example, my uncle had a parrot that learned to say "hello" and "goodbye." That made me wonder if it\'s a way for them to interact with their environment. Perhaps they imitate sounds to fit in with their flock or to get attention from humans.\n\nWait, there\'s also the aspect of survival. Do parrots mimic sounds for protection? Maybe they use it as a way to warn others about predators or to locate food sources. But I\'m not sure how that directly relates to talking like humans.\n\nAnother angle: cognitive abilities. Parrots are considered highly intelli

In [5]:
class AnimeDataLoader:
    def __init__(self, original_csv: str, processed_csv: str):
        self.original_csv = original_csv
        self.processed_csv = processed_csv

    def load_and_process(self):
        df = pd.read_csv(
            self.original_csv,
            encoding="utf-8",
            on_bad_lines="skip"
        ).dropna()

        required_cols = {"Name", "Genres", "sypnopsis"}
        if not required_cols.issubset(df.columns):
            raise ValueError("Missing required columns in CSV")

        df["combined_info"] = (
            "Title: " + df["Name"]
            + " Overview: " + df["sypnopsis"]
            + " Genres: " + df["Genres"]
        )

        df[["combined_info"]].to_csv(
            self.processed_csv,
            index=False,
            encoding="utf-8"
        )

        return self.processed_csv


In [6]:
kunal = AnimeDataLoader(
    original_csv="data/anime_with_synopsis.csv",
    processed_csv="data/processed_anime.csv"
)

kunal.load_and_process()

'data/processed_anime.csv'

In [8]:
class VectorStoreBuilder:
    def __init__(self, csv_path: str, persist_dir: str = "chroma_db"):
        self.csv_path = csv_path
        self.persist_dir = persist_dir
        self.embedding = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2"
        )

    def build_and_save_vectorstore(self):
        loader = CSVLoader(
            file_path=self.csv_path,
            encoding="utf-8",
            metadata_columns=[]
        )

        documents = loader.load()

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=0
        )
        chunks = splitter.split_documents(documents)

        db = Chroma.from_documents(
            chunks,
            self.embedding,
            persist_directory=self.persist_dir
        )
        db.persist()

    def load_vector_store(self):
        return Chroma(
            persist_directory=self.persist_dir,
            embedding_function=self.embedding
        )


In [9]:
print("Building vector database...")

original_csv = "data/anime_with_synopsis.csv"
processed_csv = "data/anime_updated.csv"
persist_dir = "chroma_db"

loader = AnimeDataLoader(original_csv, processed_csv)
processed_csv_path = loader.load_and_process()

vector_builder = VectorStoreBuilder(
    processed_csv_path,
    persist_dir=persist_dir
)
vector_builder.build_and_save_vectorstore()

print("Vector database ready.")


Building vector database...


d:\ProjectNLP\Anime\animeee\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kunal\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regul

Vector database ready.


C:\Users\kunal\AppData\Local\Temp\ipykernel_10756\3258009850.py:29: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


RETRIVER TOOL

In [10]:
vector_builder = VectorStoreBuilder(
    csv_path="",
    persist_dir="chroma_db"
)

retriever = vector_builder.load_vector_store().as_retriever()


C:\Users\kunal\AppData\Local\Temp\ipykernel_10756\3258009850.py:32: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  return Chroma(


In [11]:
query= "tell me about naruto"
retriever.invoke(query)

[Document(metadata={'source': 'data/anime_updated.csv', 'row': 10}, page_content="combined_info: Title: Naruto Overview: oments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrous beast inside the newborn Naruto. Now, Naruto is a hyperactive and knuckle-headed ninja still living in Konohagakure. Shunned because of the Kyuubi inside him, Naruto struggles to find his place in the village, while his burning desire to become the Hokage of Konohagakure leads him not only to some great new friends, but also some deadly foes. Genres: Action, Adventure, Comedy, Super Power, Martial Arts, Shounen"),
 Document(metadata={'source': 'data/anime_updated.csv', 'row': 43}, page_content="combined_info: Title: D.C.: Da Capo Overview: Every year the flowers bloom. My

In [12]:
@tool
def anime_retriever_tool(query: str) -> str:
    """
    Use this tool to search the anime knowledge base.

    Always call this tool for anime-related questions such as:
    recommendations, similarity search, genres, or plot summaries.

    Input:
    - query: User's anime preference or question.

    Output:
    - Relevant anime information retrieved from the vector database.
    """
    docs = retriever.invoke(query)
    return "\n\n".join(doc.page_content for doc in docs)


In [13]:
model_with_tools = model.bind_tools([anime_retriever_tool])

In [18]:
messages = [
    {"role": "user", "content": "Can you suggest me some anime like Naruto?"}
]

# Step 1: Model decides tool usage
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools
for tool_call in ai_msg.tool_calls:
    tool_result = anime_retriever_tool.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)


Here are some anime recommendations based on your interest in *Naruto*:

---

### **1. Naruto**  
**Genres:** Action, Adventure, Comedy, Super Power, Martial Arts, Shounen  
**Overview:** Follows Naruto Uzumaki, a young ninja who dreams of becoming the Hokage. The story blends intense battles, growth, friendship, and a richly built ninja world.  

---

### **2. Tsubasa Chronicle**  
**Genres:** Action, Adventure, Fantasy, Magic, Romance, Supernatural  
**Overview:** A dimension-hopping adventure where Syaoran travels through worlds to save his friend Sakura. Combines mystery, magical elements, and a quest-driven narrative.  

---

### **3. Kono Minikuku mo Utsukushii Sekai**  
**Genres:** Sci-Fi, Comedy, Drama, Magic, Romance, Ecchi  
**Overview:** A part-time bike courier discovers a mysterious girl and gets entangled in a magical battle. Mixes light-hearted humor with sci-fi and action.  

---

If you want more recommendations, let me know your preferred genre or theme! 😊
